# DeepLabCut for Pupil Tracking

To save a copy of this file to your Drive:
**[File > Save a copy in Drive]**

To run this notebook, simply follow the instructions and and run the cells. You should already have a project folder with labeled data in your google drive.

## Initial Set Up
First, go to Runtime > Change Runtime Type > select "GPU".

In [ ]:
#Install deeplabcut
!pip install deeplabcut

In [1]:
!module load cuda/10.0
!module load anaconda/3/2019.03
!module load tensorflow/gpu/1.14.0

**Click "RESTART RUNTIME" before continuing!**

In [1]:
#Use TensorFlow 1.x (this only works on COLAB):
#  %tensorflow_version 1.x

import os
os.environ["DLClight"]="True"   #This will surpress the GUI as it won't work on the cloud

#import deeplabcut
import deeplabcut

/u/fklotzsche/conda-envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/u/fklotzsche/conda-envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/u/fklotzsche/conda-envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/u/fklotzsche/conda-envs/DLC-GPU/lib/python3.7/site-

True

## Link to Google Drive:

Ensure the project folder is in your google drive.

**Open the config.yaml file in the folder in your google drive, and update the project path to be set to the google drive path.**

Typically, this will look like: /content/drive/My Drive/yourProjectFolderName

In [7]:
#Run this cell and follow the authorization instructions:

# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [16]:
os.listdir('/u/fklotzsche/ptmp_link/test_dir')

['Untitled.ipynb',
 'dlc-models',
 'config.yaml',
 'videos',
 'evaluation-results',
 '.ipynb_checkpoints',
 'labeled-data',
 'training-datasets']

##Edit Project Variables

In [2]:
#Setup project variables:

#EDIT THESE ACCORDING TO YOUR PROJECT DETAILS:
ProjectFolderName = '/u/fklotzsche/ptmp_link/test_dir'
DataFolderName = '/u/fklotzsche/ptmp_link/Experiments/vMemEcc/Data/SubjectData'
VideoType = '.mp4' 

#Don't edit the video path
#If you want to work with just a specific video, you can add the name to the 
#end of the path, otherwise the current path will allow all videos in the 
#folder to be analyzed
videofile_path = ['/content/drive/My Drive/'+ProjectFolderName+'/videos/'] 
videofile_path

#This creates a path variable that links to your google drive copy
#Do not edit this
path_config_file = ProjectFolderName+'/config.yaml'
path_config_file

'/u/fklotzsche/ptmp_link/test_dir/config.yaml'

## Create a Training Data Set

Once this code cell is run, the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network e.g. **pose_cfg.yaml**.


In [ ]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

## Train Network
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1, displayiters=1000,saveiters=25000)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#either of the two will return what looks like an error message, but that is expected and you can continue on to Evaluate the Network....

**When you hit "STOP" you will get a KeyInterrupt "error" - that is fine.**

## Evaluate Network
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file, plotting=True)

## Analyzing Videos
This function analyzes the videos.

The results are stored in a csv file in the same directory where the video is. 

In [ ]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType, save_as_csv=True)

Use the following instead to run on data from single subjects in the "original" vMemEcc folder structure:


In [ ]:
for subid in ['VME_S02', 'VME_S03']:
    #subid = 'VME_S13' 
    path_sub = os.path.join(DataFolderName, subid, 'EyeTracking') 
    dirs_blocks = os.listdir(path_sub) 
    for dir in dirs_blocks:
#      if not 'Block6' in dir:
#       continue
      vids_path = [os.path.join(path_sub, dir, '000')]
      print(vids_path)
      deeplabcut.analyze_videos(path_config_file,vids_path, videotype=VideoType, save_as_csv=True)

In [24]:
os.listdir('/u/fklotzsche/ptmp_link/Experiments/vMemEcc/Data/') 

['DataMNE']

## Plot Trajectories
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
# Updating the plotting within the config.yaml file:
#this doesnt need to be done if they have been changed when creating the file
#dotsize: size of the dots
#pcutoff: likelihood threshold

#config_path = path_config_file
#edits = {'dotsize': 2,
#          'pcutoff': 0.8}
#deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

In [ ]:
#filter predictions:
#deeplabcut.filterpredictions(config_path, videofile_path, videotype=VideoType)

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType, filtered=False)

## Create Labeled Video
This creates a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file, videofile_path, videotype=VideoType, filtered=False, codec='jpeg')

In [ ]:
subid = 'VME_S04'
path_sub = os.path.join(DataFolderName, subid, 'EyeTracking') 
dirs_blocks = os.listdir(path_sub) 
for dir in dirs_blocks[2:]:
  vids_path = [os.path.join(path_sub, dir, '000')]
  print(vids_path)
  deeplabcut.create_labeled_video(path_config_file, vids_path, videotype=VideoType, filtered=False, codec='jpeg')

#Optional

## Downsample Video

The video should already be downsampled if you used kmeans as the clustering method, however you can use this to downsample if need be.

Downsample the video by running the following line of code

In [ ]:
deeplabcut.DownSampleVideo(videofile_path, width=-1, height=300, outsuffix='downsampled', outpath=None, rotateccw=False)

## Add New Videos

In [ ]:
#to add more videos
deeplabcut.add_new_videos(config_path,[‘full path of video 4’, ‘full path of video 5’],copy_videos=True/False)

## Anlayze Skeleton

In [ ]:
deeplabcut.analyzeskeleton(path_config_file, videofile_path, videotype=VideoType, save_as_csv=True)